![Podmanlogo](Pictures/podman-logo.png)

# Podman ecosystem

We have seen what an awsome piece of software Podman is, but that is not all. Podman counts with some other tools in its ecosystem that makes everything run smoothly.
Lets review all of them.

## Managing multiple containers: podman compose and kubernetes

Running a single container with podman is just fine, we have demonstrated how simple it is througout this workshop. But, what happens if I want to run multiple containers simultaneously? Do you need to execute them one by one?

You can use the tool podman compose to execute multiple containers in a single command. Podman compose is a tool that helps you defining the list of containers and all of their configuration in a text file written in yaml format. Then you can choose to execute, stop, remove, update all of the containers defined in the file at once. Podman compose is a very powerful tool but it's limited to execute on a single system.

Lets learn how podman-compose can help you deploying multiple containers at once. First we are going to build two containers called "container-A" and "container-B", for this we will use the same Containerfile that we used in the section 2 of this workshop. But this time the web service will answer "I'm SERVICE A running in container-A" when doing a curl to container-A and "I'm SERVICE B running in container-B" when running curl against container-B.

In [ ]:
%login student491@16.31.86.187

In [8]:
!echo "I'm SERVICE A running in container-A" > indexA.html
!echo "I'm SERVICE B running in container-B" > indexB.html
!echo -e "FROM docker.io/redhat/ubi9\nRUN dnf -y update && dnf -y install httpd  && dnf clean all\nCOPY indexA.html /var/www/html/index.html\nEXPOSE 80\nENTRYPOINT /usr/sbin/httpd -DFOREGROUND" > ContainerfileA
!echo -e "FROM docker.io/redhat/ubi9\nRUN dnf -y update && dnf -y install httpd  && dnf clean all\nCOPY indexB.html /var/www/html/index.html\nEXPOSE 80\nENTRYPOINT /usr/sbin/httpd -DFOREGROUND" > ContainerfileB
!podman build --tag container-a:v1 -f ./ContainerfileA
!podman build --tag container-b:v1 -f ./ContainerfileB

STEP 1/5: FROM docker.io/redhat/ubi9
STEP 2/5: RUN dnf -y update && dnf -y install httpd  && dnf clean all
--> Using cache 424ba23c0a017c416294322d6c46ae63afb8f2901f6396b4c3e23a984e511e63
--> 424ba23c0a01
STEP 3/5: COPY indexA.html /var/www/html/index.html
--> Using cache 73a3f5a96ae806c772a543e1f4a4db2d8c2dc11f223c4cc7ac8f9bd43f4702a7
--> 73a3f5a96ae8
STEP 4/5: EXPOSE 80
--> Using cache a7692d93b4ad9b3057651933614dac0927c0105884ac5421635f7a42e3d82126
--> a7692d93b4ad
STEP 5/5: ENTRYPOINT /usr/sbin/httpd -DFOREGROUND
--> Using cache cf4d05b39b49a4f18cf3b52d7e6c76c90288a3436434aabb04f126a84090fa96
COMMIT container-a:v1
--> cf4d05b39b49
Successfully tagged localhost/container-a:v1
cf4d05b39b49a4f18cf3b52d7e6c76c90288a3436434aabb04f126a84090fa96
STEP 1/5: FROM docker.io/redhat/ubi9
STEP 2/5: RUN dnf -y update && dnf -y install httpd  && dnf clean all
--> Using cache 424ba23c0a017c416294322d6c46ae63afb8f2901f6396b4c3e23a984e511e63
--> 424ba23c0a01
STEP 3/5: COPY indexB.html /var/www/html/i

Check both your images have been created:

In [9]:
!podman images

REPOSITORY                                                          TAG         IMAGE ID      CREATED             SIZE
localhost/container-b                                               v1          6132a679a311  About a minute ago  241 MB
localhost/container-a                                               v1          cf4d05b39b49  About a minute ago  241 MB
<none>                                                              <none>      95f0b9bd1c87  7 days ago          314 MB
docker.io/redhat/ubi9                                               latest      20cef057605e  8 days ago          217 MB
quay.io/skupper/skupper-router                                      2.4.3       9f6d24d58351  2 weeks ago         275 MB
registry.fedoraproject.org/fedora                                   latest      a1cd3cbf8ada  3 weeks ago         195 MB
localhost/bcl-ov                                                    15          ef3ba4fe4c30  7 weeks ago         717 MB
<none>                            

Now, we can create a compose file and create both containers at once. First lets create the file and then we will see what's in it.

In [12]:
!echo -e "services:\n  service-A:\n      image: localhost/container-a:v1\n      container_name: container-A\n      restart: always\n      ports:\n        - "8080:80"\n\n  service-B:\n      image:     localhost/container-b:v1\n      container_name: container-B\n      restart: always\n      ports:\n        - "8081:80"\n      depends_on:\n        - service-A\n" > compose.yml
!cat compose.yml

services:
  service-A:
      image: localhost/container-a:v1
      container_name: container-A
      restart: always
      ports:
        - 8080:80

  service-B:
      image:     localhost/container-b:v1
      container_name: container-B
      restart: always
      ports:
        - 8081:80
      depends_on:
        - service-A



As you can see the file that we just created is using the Yaml format, meaning that indentation defines what objects are at the same level or what lines are options inside certain object.

In our example, as you can see in the output of previous command, you can see that first we have a line that states "services" and then we have two lines "service-A" and "service-B", as the last two lines are indented inside the "services" section podman-compose will know that they are services objects. Then we have a few options within each service that define how they will need to be deployed. Both "service-A" and "service-B" are arbitrary names.

Within each service we define the container image to be used with the "image" option, the container name with "container_name" option, the ports to be exposed with the "ports" option and the restart policy with the "restart" option.

In the service-B service we defined the option "depends_on" stating that service-B will only be started once service-A has started. In our application this doesn't matter, but many applications will have services that depend on other services.

Now we can just run the command to execute both containers.

In [14]:
!podman-compose up -d

podman-compose version: 1.0.6
['podman', '--version', '']
using podman version: 4.7.0
** excluding:  set()
['podman', 'ps', '--filter', 'label=io.podman.compose.project=wkshp-podman101', '-a', '--format', '{{ index .Labels "io.podman.compose.config-hash"}}']
['podman', 'network', 'exists', 'wkshp-podman101_default']
podman run --name=container-A -d --label io.podman.compose.config-hash=1d18b6a15a9a1c69b622c3cf88150b6abaa8d6edbe6a886bcc5ac80848a912b8 --label io.podman.compose.project=wkshp-podman101 --label io.podman.compose.version=1.0.6 --label PODMAN_SYSTEMD_UNIT=podman-compose@wkshp-podman101.service --label com.docker.compose.project=wkshp-podman101 --label com.docker.compose.project.working_dir=/home/ppreciad/Github/wod-notebooks/WKSHP-Podman101 --label com.docker.compose.project.config_files=compose.yml --label com.docker.compose.container-number=1 --label com.docker.compose.service=service-A --net wkshp-podman101_default --network-alias service-A -p 8080:80 --restart always loca

Now check both of your containers are running.

In [15]:
!podman ps

CONTAINER ID  IMAGE                     COMMAND     CREATED         STATUS         PORTS                 NAMES
c4c83c622351  localhost/container-a:v1              28 seconds ago  Up 28 seconds  0.0.0.0:8080->80/tcp  container-A
43f133c259ac  localhost/container-b:v1              27 seconds ago  Up 27 seconds  0.0.0.0:8081->80/tcp  container-B


You see how easy it is to create multiple containers with podman-compose. Now lets stop both of them.

In [16]:
!podman-compose down

podman-compose version: 1.0.6
['podman', '--version', '']
using podman version: 4.7.0
** excluding:  set()
podman stop -t 10 container-B
container-B
exit code: 0
podman stop -t 10 container-A
container-A
exit code: 0
podman rm container-B
container-B
exit code: 0
podman rm container-A
container-A
exit code: 0


Check they have been stopped.

In [17]:
!podman ps

CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES


In this example we have used using two web server containers, but think about how useful this tool would be for a classic three tier app. For example one having a frontend, a backend and a database. You could deploy all of the pieces in the right order in a super easy and repeatable way with podman-compose.

Podman-compose is a very interesting tool, but it's limited to deploying containers to a single system. If you're looking at deploying your containers in multiple systems simultaneously in order to get scalability and high availability then you will most probably end up using kubernetes as it has become the standard platform for running containerized workloads in the enterprise space.
Kubernetes, same as podman composer, uses yaml files to define the desired state of your containerized workloads and, once you pass the file to the control plane nodes of kubernetes, it will make sure your containers run according to your specifications. Furthermore they will be spread accross all the nodes that are part of your kubernetes cluster.

![k8slogo](Pictures/kubernetes-logo.png)

Now, transitioning from single system container engines like Podman or Docker to kubernetes may not always be easy. That is why Podman can generate a kubernetes yaml file from the container that you are running righ now with Podman. First you just need to execute a new container:

In [ ]:
podman run -d --rm --name=kubepodman docker.io/redhat/ubi9 sleep 999

Now your container is running:

In [ ]:
podman ps

With this, you can execute the "podman generate kube" command to export a yaml file, which you can use as definition for your kubernetes environment.

In [ ]:
podman generate kube kubepodman > kubepodman.yml
cat kubepodman.yml

The output of the previous command is the content of the yaml file that you need to use for running your container in a kubernetes workload. At the beginning of the file you can even see a few comments in which you find the command you need to run in order to start executing this container in your kubernetes cluster: "kubectl create -f kubepodman.yml".

As you can see, podman has multiple tools that allow you to transition to managing multiple containers at once and simplify the transition to those tools.

## Advanced container images management: Skopeo

Skopeo is a tool for manipulating, inspecting, signing, and transferring container images and image repositories on linux systems, Windows and MacOS. Like Podman and Buildah, Skopeo is an open source community-driven project that does not require running a container daemon.

With Skopeo, you can inspect images on a remote registry without having to download the entire image with all its layers, making it a lightweight and modular solution for working with container images across different formats, including Open Container Initiative (OCI) and Docker images.

![skopeologo](Pictures/skopeo.png)

In most cases you will find Skopeo a super useful tool to manage container images that are stored in remote registries without pulling them to your system. For example you can inspect a remote container by using the command "skopeo inspect". First lets make sure that the image we will be inspecting is not locally stored in our system.

In [ ]:
podman stop --all
podman rmi docker.io/redhat/ubi9
podman images

You should see an empty list.

Now lets inspect a remote container image, an image residing in docker.io, with skopeo:

In [ ]:
skopeo inspect docker://docker.io/redhat/ubi9-minimal

Pay attention to the naming convention, you need to specify "docker://" before the repo, container name and tag.

As you can see we are able to get the information about the container image but we did not need to pull the image for that. Lets check that the image was not downloaded.

In [ ]:
podman images

Another very intereting and widely used skopeo functionality is copying images from a source registry to a target registry with the "skopeo copy" command, by using it you don't need to have a copy of the image in your local system.

## Advanced container image creation using buildah

The command line tool buildah provides the ability of creating container images from a running container, a Containerfile or from scratch. The resulting images are OCI (Open Container Initiative) compliant, meaning that they will work with any container enginge that complies with the OCI standard such as podman, docker or CRI-O.

![buildahlogo](Pictures/buildah.png)

Most enterprises will be able to just work with Podman build capabilities, but if you need to cover an advanced use case like building a container image from scratch, that is without a base container image, buildah is definitely the tool you're looking for. We will not cover this use case in this workshop as it's quite advanced.

## Podman for Desktop

If you're not experienced with command line tools there is a super powerful tool that brings a graphical user experience to Podman, that is Podman Desktop. It is available for Windows, macOS and Linux.

![PodmanDesktop](Pictures/Podman_Desktop.png)

Moreover, Podman Desktop is not limited to Podman, it can also interact with kubernetes clusters and is a very helpful tool in the transition to containerized applications for enterprises that are willing to take this path.

# Cleanup

In [19]:
!podman stop --all
!podman rm --all
!podman rmi docker.io/redhat/ubi9 localhost/container-a:v1 localhost/container-a:v1
!rm kubepodman.yml indexA.html indexB.html ContainerfileA ContainerfileB compose.yml

Error: 3 errors occurred:
	* docker.io/redhat/ubi9: image not known
	* localhost/container-a:v1: image not known
	* localhost/container-a:v1: image not known
rm: cannot remove 'kubepodman.yml': No such file or directory
rm: cannot remove 'indexA.html': No such file or directory
rm: cannot remove 'indexB.html': No such file or directory
rm: cannot remove 'ContainerfileA': No such file or directory
rm: cannot remove 'ContainerfileB': No such file or directory


In [ ]:
%logout